In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/agents-intensive-capstone-project/Hackathon dataset.txt
/kaggle/input/drug-drug-interactions/db_drug_interactions.csv


In [3]:
DATA_PATH = "/kaggle/input/drug-drug-interactions/db_drug_interactions.csv"
df = pd.read_csv(DATA_PATH)

In [4]:
df.head()

,Drug 1,Drug 2,Interaction Description
0,Trioxsalen,Verteporfin,Trioxsalen may increase the photosensitizing a...
1,Aminolevulinic acid,Verteporfin,Aminolevulinic acid may increase the photosens...
2,Titanium dioxide,Verteporfin,Titanium dioxide may increase the photosensiti...
3,Tiaprofenic acid,Verteporfin,Tiaprofenic acid may increase the photosensiti...
4,Cyamemazine,Verteporfin,Cyamemazine may increase the photosensitizing ...


In [5]:
df.columns

Index(['Drug 1', 'Drug 2', 'Interaction Description'], dtype='object')

In [6]:
# CONFIG
DRUG_A_COL = "Drug 1"
DRUG_B_COL = "Drug 2"
DESCRIPTION_COL = "Interaction Description"
# dataset does not include severity ratings
SEVERITY_COL = None  


# Normalization Helper
def normalize_drug_name(name: str) -> str:
    if not isinstance(name, str):
        return ""
    return name.strip().lower()

In [7]:
from collections import defaultdict

# Normalize columns
df["_drug_a_norm"] = df[DRUG_A_COL].apply(normalize_drug_name)
df["_drug_b_norm"] = df[DRUG_B_COL].apply(normalize_drug_name)

# Remove invalid entries
df = df[(df["_drug_a_norm"] != "") & (df["_drug_b_norm"] != "")].copy()

# Build lookup: key = frozenset({drug1, drug2})
interaction_lookup = defaultdict(list)

for _, row in df.iterrows():
    a = row["_drug_a_norm"]
    b = row["_drug_b_norm"]
    key = frozenset([a, b])

    record = {
        "drug_a": row[DRUG_A_COL],
        "drug_b": row[DRUG_B_COL],
        "description": row[DESCRIPTION_COL]
    }
    
    interaction_lookup[key].append(record)

len(interaction_lookup)

191135

In [8]:
def tool_check_pair(drug1: str, drug2: str):
    """
    Return interaction records for a pair of drugs.
    """
    d1 = normalize_drug_name(drug1)
    d2 = normalize_drug_name(drug2)
    key = frozenset([d1, d2])
    return interaction_lookup.get(key, [])

In [9]:
def tool_find_interactions_for_list(drug_list):
    """
    Check all unique pairs within a list of drugs.
    """
    normalized = [normalize_drug_name(d) for d in drug_list]
    results = []
    
    n = len(normalized)
    for i in range(n):
        for j in range(i+1, n):
            d1 = normalized[i]
            d2 = normalized[j]
            key = frozenset([d1, d2])
            interactions = interaction_lookup.get(key, [])
            
            if interactions:
                results.append({
                    "drug1_input": drug_list[i],
                    "drug2_input": drug_list[j],
                    "drug1_norm": d1,
                    "drug2_norm": d2,
                    "interactions": interactions
                })
    return results

In [10]:
def tool_format_report(drug_list):
    """
    Create a human-friendly report summarizing interactions.
    """
    interactions = tool_find_interactions_for_list(drug_list)

    if not interactions:
        return (
            "### No interactions found in this dataset.\n"
            "**Important:** Absence of data is NOT confirmation of safety.\n"
            "Consult a pharmacist or doctor for medical decisions."
        )

    lines = []
    lines.append("## Drug–Drug Interaction Report")
    lines.append("### Drugs entered: " + ", ".join(drug_list))
    lines.append("")

    for entry in interactions:
        d1 = entry["drug1_input"]
        d2 = entry["drug2_input"]

        lines.append(f"### Interaction: **{d1}** × **{d2}**")

        for rec in entry["interactions"]:
            desc = rec.get("description") or "No description provided."
            lines.append(f"- **Detail**: {desc}")

        lines.append("")

    lines.append("---")
    lines.append(
        "*This is an educational tool only. It uses a dataset and may not cover all interactions.*"
    )

    return "\n".join(lines)

In [11]:
test_list = ["Aspirin", "Warfarin", "Ibuprofen"]
print(tool_format_report(test_list))

## Drug–Drug Interaction Report
### Drugs entered: Aspirin, Warfarin, Ibuprofen

### Interaction: **Warfarin** × **Ibuprofen**
- **Detail**: Warfarin may increase the anticoagulant activities of Ibuprofen.

---
*This is an educational tool only. It uses a dataset and may not cover all interactions.*
